# Do two things:

* Convert filenames to include the date
* Add a raw cell to the top of notebooks with the jekyll header

In [80]:
from glob import glob
import os
import shutil as sh
import nbformat as nbf
from tqdm import tqdm
from copy import deepcopy

In [82]:
sh.rmtree('content/')
sh.copytree('../choldgraf.github.io/content/', 'content/')

path_notebooks_new = './notebooks/'
ipynb_files = glob('content/articles/**/*.ipynb', recursive=True)

for ifile in tqdm(ipynb_files):
    file_name = os.path.basename(ifile)
    file_path = os.path.dirname(ifile)
    
    # Parse metadata
    meta_path = file_name + '-meta'
    with open(os.path.join(file_path, meta_path), 'r') as ff:
        lines = ff.readlines()
        lines = [ii.strip() for ii in lines]
        meta = {ii.split(':')[0]: ii.split(':')[1] for ii in lines}
    
    year = meta['Date'].strip().split('-')[0]
    use_data = {'date': meta['Date'].strip().split(' ')[0],
            'category': meta['Category'].strip(),
            'tags': meta['Tags'].strip(),
            'permalink': meta['Slug'].strip(),
            'title': meta['Title'].strip()}

    if meta.get('Interactive'):
        use_data['interactive'] = meta['Interactive'].strip()

    new_name = file_name
    if '-' in new_name:
        new_name = '-'.join(new_name.split('-')[1:])
    new_name = use_data['date'] + '-' + new_name

    # This is the metadata cell
    cell_source = ['---']
    for key, val in use_data.items():
        cell_source.append('{}: {}'.format(key, val))
    cell_source.append('---')
    cell_source = '\n'.join(cell_source)

    # Read in notebook
    new_cell = nbf.NotebookNode(cell_type='raw', source=cell_source, metadata={})
    ntbk = nbf.read(ifile, nbf.NO_CONVERT)
    ntbk.cells.insert(0, new_cell)
    
    nbf.write(ntbk, ifile)
    if not os.path.exists(os.path.join(path_notebooks_new, year)):
        os.makedirs(os.path.join(path_notebooks_new, year))
    sh.move(ifile, os.path.join(path_notebooks_new, year, new_name))
    os.remove(os.path.join(file_path, meta_path))
    
# Remove ipynb checkpoints
for ifolder in glob('../content/**/.ipynb_checkpoints', recursive=True) + glob('../content/**/html', recursive=True):
    sh.rmtree(ifolder)


100%|██████████| 13/13 [00:02<00:00,  5.48it/s]
